# TP 5 - Chiffrement asymétrique

In [10]:
from OutilsCrypto import *

Le chiffrement asymétrique, ou chiffrement à clef publique, permet d'assurer la confidentialité d'une communication, ou d'authentifier les participants, sans que cela repose sur une donnée secrète partagée entre ceux-ci, contrairement à la cryptographie symétrique qui nécessite de partager ce secret au préalable.

Le terme asymétrique s'explique par le fait que le chiffrement est basé sur 2 clés : la clé <b>publique</b> utilisée pour <b>chiffrer</b> le message et la clé <b>privée</b> utilisée pour <b>déchiffrer</b> le message reçu.

Imaginons que Alice souhaite envoyer un message confidentiel à Bob. Bob va commencer par créer un tel couple de clé et publie sa clé publique alors qu'il garde sa clé secrète pour lui. Alice va chiffrer son message avec la clé publique de Bob et celui-ci sera le seul à pouvoir le déchiffrer à l'aide de sa clé privée.

Ce concept a été créé par Whitfield Diffie et Martin Hellman en 1976 pour écrire un algorithme permettant d'échanger la clé privée d'un algorithme de chiffrement symétrique et a ensuite été mis en oeuvre dans un algorithme de chiffrement asymétrique par Ronald <b>R</b>ivest, Adi <b>S</b>hamir et Leonard <b>A</b>dleman en 1978. Cet algorithme, plus connu sous le nom de RSA,  est encore très utilisé de nos jours.

## 1. Echange de clé de Diffie-Hellman

L'algorithme d'échange de clé de Diffie-Hellman est utilisé notamment lors de l’ouverture d’une connexion à un site sécurisé via le protocole SSL/TLS.

L'algorithme d'échange de clé de Diffie-Hellman permet à Alice et Bob de se mettre d'accord secrètement sur un nombre qui pourra servir par la suite de clé privée dans un système de chiffrement symétrique.

Le principe est le suivant :
- Alice et Bob ont choisi un nombre premier $p$ et un entier $g$ strictement plus petit que $p$ (ces 2 nombres n'ont pas besoin d'être secrets) ;
- Alice choisit un nombre $a$ au hasard qu'elle garde secret, et envoie à Bob le nombre $\displaystyle A=g^{a}[p]$ (« g puissance a modulo p ») ;
- De même, Bob choisit un nombre $b$ au hasard qu'il garde secret, et transmet le nombre $\displaystyle B=g^{b}[p]$ ;
- Alice, avec le nombre $B$ reçu de Bob, calcule $\displaystyle B^{a}[p]$. Elle obtient donc le nombre $\displaystyle g^{ba}[p]$ ;
- Bob fait le calcul analogue avec le nombre $A$ reçu d'Alice : $\displaystyle A^{b}[p]$. Il obtient $\displaystyle g^{ab}[p]$, ce qui est le même nombre que celui obtenu par Alice.

Avec cet algorithme, Alice et Bob se retrouvent avec la connaissance du même nombre $g^{ab}[p]$ qui pourra leur servir de clé secrète pour chiffrer une communication à l'ai de d'un algorithme de chiffrement symétrique.

Quelqu'un qui intercepterait $A$ et/ou $B$ n'aurait pas la possibilité de deviner la clé $g^{ab}[p]$, car il n'existe pas à ce jour d'algorithme performant capable de faire ce calcul en temps raisonnable (à condition que les nombres $p$, $g$, $a$ et $b$ soient choisis de taille conséquente).

<b>Exercice 1.</b> Écrire la fonction <code>echangeDiffieHellman(p,g)</code> qui simule cet échange en demandant à Alice et Bob leurs nombres secrets et en affichant la clé générée pour Alice et pour Bob. Vérifiez que c'est bien la même clé.

In [5]:
def calculer_cle_privee(nombre_secret, base, modulo):
    return (base**nombre_secret) % modulo


def echangeDiffieHellman(p, g):
    alice_secret = int(input("Alice, entrez le nombre secret : "))
    bob_secret = int(input("Bob, entrez le nombre secret : "))

    A = calculer_cle_privee(alice_secret, g, p)
    B = calculer_cle_privee(bob_secret, g, p)

    cle_alice = calculer_cle_privee(alice_secret, B, p)
    cle_bob = calculer_cle_privee(bob_secret, A, p)

    print("Clé Alice : " + str(cle_alice))
    print("Clé Bob : " + str(cle_bob))

    if cle_alice == cle_bob:
        print("Les clés sont égales")
    else:
        print("Les clés ne sont pas égales")

Exécuer le bloc suivant pour vérifier votre fonction.

In [7]:
echangeDiffieHellman(23,5) # si Alice choisit a = 6 et Bob choisit b = 15, la clé générée doit être égale à 2

Clé Alice : 4
Clé Bob : 4
Les clés sont égales


## 2. RSA

L'algorithme de chiffrement RSA utilise un couple (clé publique, clé privée) construit de la façon suivante :
- on choisit 2 nombres premiers $p$ et $q$
- on calcule $n = pq$
- on calcule $\varphi(n) = (p-1)(q-1)$
- on choisit un entier $e$ premier avec $\varphi(n)$ ($2\leq e\leq \varphi(n)$)
- on calule l'inverse $d$ de $e$ modulo $\varphi(n)$.

Le couple $(e,n)$ est alors la clé publique et le couple $(d,n)$ est la clé privée.

Si Alice veut envoyer un message à Bob, alors elle va le chiffrer avec la clé publique de Bob et Bob déchiffrera le massage reçu avec sa clé privée.

Le chiffrement se fait de la façon suivante :
- chaque lettre est codée en un nombre $x$ entre 0 et 25
- puis on effectue le chiffrement en calculant $X \equiv x^e \pmod{n}$

Le déchiffrement se fait de la façon suivante :
- on effectue le déchiffrement de chaque nombre $X$ reçu en calculant $x \equiv X^d \pmod{n}$
- puis on décode chaque résultat pour retrouver la lettre associée

Le message est bien déchiffré grâce à la propriété suivante :

Soient $p$ et $q$ deux nombres premiers et soit $n = pq$. Soit $a$ un entier tel que $0\leq a \leq n$, alors
$$
\forall k\in\mathbb{N},\  a^{k\varphi(n)+1} \equiv a \pmod n
$$

En effet, $X^d \equiv (x^{e})^d \equiv x^{ed} \equiv x^{k\varphi(n)+1} \equiv x \pmod n$ et on retrouve donc bien le message $x$ d'origine.


<b>Exercice 2.</b> Écrire la fonction <code>get_prime(size)</code> qui renvoie un nombre premier au hasard de <code>size</code> bits.

<i>(vous pouvez par exemple utiliser la fonction <code>random.getrandbits(size)</code> de la librairie <code>math</code> et tester si le nombre obtenu est premier à l'aide de la fonction que vous aviez codée dans le TP 1)</i>

In [20]:
import random

def get_prime(size):
    while(True):
      nb = random.getrandbits(size)
      nb |= 1 # force le dernier bit à 1 pour être sûr d'avoir un nombre impair
      if(isPrime(nb)):
        return nb

Tester votre fonction en exécutant le bloc suivant.

In [21]:
get_prime(8)

151

Écrire la fonction <code>chiffre_RSA(txt, size)</code> qui renvoie <code>txt_chiffre, (e, n), (d, n)</code> où <code>txt_chiffre</code> est le chiffrement RSA de <code>txt</code> sous la forme d'une liste de nombres, <code>(e,n)</code> est la clé publique générée et <code>(d,n)</code> est la clé privée générée.

<div style="background-color:rgba(255, 255, 0, 0.19);padding:3%;">
<b>Important : pour calculer les puissances $x^e \pmod n$, utilisez pour l'instant l'opération ** : <code>x**e %n</code>
    </b>
</div>

In [22]:
def chiffre_RSA(txt, size):
    # Générer deux nombres premiers distincts
    p = get_prime(size)
    q = get_prime(size)

    # Calculer le produit des deux nombres premiers
    n = p * q

    # Calculer la fonction d'Euler de n
    phi_n = (p - 1) * (q - 1)

    # Choisir un exposant de chiffrement e aléatoire
    e = random.randint(2, phi_n - 1)
    while not isPrime(e):
        e = random.randint(2, phi_n - 1)

    # Calculer l'exposant de déchiffrement d
    d = invMod(e, phi_n)

    # Chiffrer le texte en utilisant la clé publique (e, n)
    txt_chiffre = [pow(ord(char), e, n) for char in txt]

    return txt_chiffre, (e, n), (d, n)

Tester votre fonction en exécutant le bloc suivant.

In [35]:
(texte_chiffre, (e,n), (d,n)) = chiffre_RSA("messagesecret",8)

print("texte chiffré = ",texte_chiffre) # Affiche une liste de nombres du type [26665, 29673, ...]
print("clé publique = ", (e,n)) # Affiche un couple du type (3143, 38579) qui représente la clé publique
print("clé privée = ", (d,n)) # Affiche un couple du type (7751, 38579) qui représente la clé privée

texte chiffré =  [37157, 812, 16424, 16424, 11087, 6951, 812, 16424, 812, 543, 36875, 812, 26227]
clé publique =  (31469, 45901)
clé privée =  (4517, 45901)


Écrire la fonction <code>dechiffre_RSA(txt, d, n)</code> qui déchiffre le massage <code>txt</code> avec la clé privée <code>(d,n)</code>

In [36]:
def dechiffre_RSA(txt_chiffre, d, n):
    txt_dechiffre = "".join(chr(pow(char, d, n)) for char in txt_chiffre)
    return txt_dechiffre

Tester votre fonction en exécutant le bloc suivant.

In [40]:
(texte_chiffre, (e,n), (d,n)) = chiffre_RSA("messagesecret",8)
print("texte chiffré = ",texte_chiffre)
print("texte déchiffré = ",dechiffre_RSA(texte_chiffre, d,n))

texte chiffré =  [305, 738, 143, 143, 48, 180, 738, 143, 738, 239, 695, 738, 676]
texte déchiffré =  messagesecret


## 3. Sécurité et rapidité

La sécurité de cet algorithme repose sur la difficulté de trouver la clé privée $(d,n)$ malgré la connaissance de la clé publique $(e,n)$. Expliquer en quoi c'est difficile et comment nous devons choisir $p$ et $q$ pour garantir la sécurité du chiffrement.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : La complexité du chiffrement RSA repose sur la difficulté à factoriser le produit de deux grands nombres premiers, généralement notés p et q. Plus ces nombres sont grands, plus la factorisation devient difficile. La sécurité du chiffrement RSA repose sur le fait que factoriser le produit de deux grands nombres premiers est une tâche complexe, notamment lorsque ces nombres sont choisis aléatoirement et sont de magnitude élevée. Il faut donc choisir p et q de manière aléatoire et les rendre les plus grands possible</b>
</div>

Essayons de choisir des nombres premiers plus grands. Exécuter le bloc suivant en prenant des tailles de plus en plus grandes, quel problème rencontrons-nous ?

In [42]:
size = 8
p = get_prime(size)
print(p)
q = get_prime(size)
print(q)
n = p*q
print(n)

179
173
30967


On considèrera qu'il faudrait choisir une taille de 512 bits pour assurer un minimum de sécurité, avez-vous testé une telle valeur ? Il est temps de revoir notre façon de tester si un nombre est premier ou non...

### 3.1 Test de Miller-Rabin

L'algorithme suivant s'appuie sur le test de Miller-Rabin pour vérifier si un nombre est premier ou non.

In [43]:
def is_prime_MR(n):
  # Vérification rapide de la primalité en utilisant les tests de Miller-Rabin
  if n in (2, 3):
    return True
  if n == 1 or n % 2 == 0:
    return False
  d = n - 1
  r = 0
  while d % 2 == 0:
    d //= 2
    r += 1
  for _ in range(10):
    a = random.randrange(2, n - 1)
    x = pow(a, d, n)
    if x == 1 or x == n - 1:
      continue
    for _ in range(r - 1):
      x = pow(x, 2, n)
      if x == n - 1:
        break
    else:
      return False
  return True

<b>Exercice 3.</b> Modifier la fonction de recherche de nombre premier pour qu'elle utilise cette nouvelle fonction.

In [44]:
def get_prime_MR(size):
    while True:
      nb = random.getrandbits(size)
      nb |= 1
      if is_prime_MR(nb):
        return nb

Tester à nouveau la recherche de nombres premiers de grande taille.

In [45]:
size = 512
p = get_prime_MR(size)
print(p)
q = get_prime_MR(size)
print(q)
n = p*q
print(n)

10004842448303499734532721501211059245078590014252827084958898283310793459185962477314122870961148307967462801553429614030064563071989123345740709833524187
1063478453933329973387789071618509184421055372348545579747086534627433784382957466600475137452788954481266651395378829279749936914767425129650553914926083
10639934378768357708190897207741773669853907321854821997705324697124657554307980999053337043487160062509912906090762924332925341025762077763748431651131759817120782041107272467483185430910034334233324038315893393726289792125937202154001203133594949911669569436065647466182400109080083059826165051277497669521


Le résultat est désormais très rapide, même pour de très grands nombres premiers.
Modifier la fonction chiffreRSA avec cette nouvelle façon de générer des nombres premiers.

In [50]:
def chiffre_RSA_MR(txt, size):
    # Générer deux nombres premiers distincts
    p = get_prime_MR(size)
    q = get_prime_MR(size)

    # Calculer le produit des deux nombres premiers
    n = p * q

    # Calculer la fonction d'Euler de n
    phi_n = (p - 1) * (q - 1)

    # Choisir un exposant de chiffrement e aléatoire
    e = random.randint(2, phi_n - 1)
    while not is_prime_MR(e):
        e = random.randint(2, phi_n - 1)

    # Calculer l'exposant de déchiffrement d
    d = invMod(e, phi_n)

    # Chiffrer le texte en utilisant la clé publique (e, n)
    txt_chiffre = [pow(ord(char), e, n) for char in txt]

    return txt_chiffre, (e, n), (d, n)

Tester votre fonction en exécutant le bloc suivant avec différentes tailles.

In [53]:
size = 8
(texte_chiffre, (e,n), (d,n)) = chiffre_RSA_MR("messagesecret",size)
print("texte chiffré = ",texte_chiffre)
print("texte déchiffré = ",dechiffre_RSA(texte_chiffre, d,n))

texte chiffré =  [1686, 1138, 1857, 1857, 1626, 1701, 1138, 1857, 1138, 1248, 351, 1138, 925]
texte déchiffré =  messagesecret


Il nous reste un problème : si $p$ et $q$ sont des grands nombres, dans l'algorithme RSA, on est amené à calculer des puissances énormes, ce qui pose un gros problème si on ne fait pas ça intelligemment !

### 3.2 Exponentiation modulaire rapide

L'idée de l'algorithme d'exponentiation modulaire rapide est de ne calculer que des puissances de 2 en prenant le modulo à chaque fois.

Par exemple, pour calculer $4^{13} \mod{26}$ par exemple, on va écrire
$$
4^{13} \equiv 4*(4^2)^6 \equiv 4*((16)^2)^3 \equiv 4*(256)^3 \equiv 4*(22)^3 \equiv 4*22*22^2 \equiv 4*22*484 \equiv 4*22*16 \equiv 1408 \equiv 4 \pmod{26}
$$

L'algorithme en pseudo-code peut se décrire de la façon suivante :

<code>
fast_exp(x,k,n) # calcul de x^k mod n
    r = 1
    x = x %n
    TantQue k > 0
        Si k est impair
            r = r*x %n
        x = x^2 %n
        k = le quotient de la division de k par 2
    retourner r
</code>

<b>Exercice 4.</b> Écrire la fonction <code>fast_exp(x,k,n)</code> qui implémente l'exponentiation modulaire rapide.

In [59]:
def fast_exp(x,k,n):
    r = 1
    x = x % n

    while k > 0:
        if k % 2 == 1:
            r = (r * x) % n
        x = (x * x) % n
        k //= 2
    return r

Tester votre fonction en exécutant le bloc suivant avec différents paramètres.

In [19]:
fast_exp(5,6,4)

1

Modifier les fonctions chiffre_RSA et dechiffre_RSA avec cette nouvelle façon de calculer les puissances.

In [60]:
def chiffre_RSA_MR_FE(txt, size):
    p = get_prime_MR(size)
    q = get_prime_MR(size)

    n = p * q
    phi_n = (p - 1) * (q - 1)

    e = random.randint(2, phi_n)
    while not is_prime_MR(e):
        e = random.randint(2, phi_n)
    d = invMod(e, phi_n)

    txt_chiffre = [fast_exp(ord(char), e, n) for char in txt]

    return txt_chiffre, (e, n), (d, n)


def dechiffre_RSA_FE(txt_chiffre, d, n):
    txt_dechiffre = "".join(chr(fast_exp(char, d, n)) for char in txt_chiffre)
    return txt_dechiffre

Tester votre fonction en exécutant le bloc suivant avec différentes tailles.

In [61]:
(texte_chiffre, (e, n), (d, n)) = chiffre_RSA_MR_FE("messagesecret", 512)
dechiffre_RSA_FE(texte_chiffre, d, n)

'messagesecret'

## 4. Authentification

<b>Exercice 5.</b> Le chiffrement RSA peut également servir à authentifier l'expéditeur d'un message. En effet, il suffit à l'expéditeur de chiffrer son message à l'aide de sa clé privée. Écrire dans le bloc suivant la simulation d'un envoie de message de Alice à Bob de sorte que le message soit à la fois authentifié comme venant bien d'Alice et chiffré pour garder son contenu secret.

In [63]:
# Non fonctionnel
size = 8
messageOriginal = "Salut Bob c'est Alice"

messageChiffre, priveAlice, publicAlice = chiffre_RSA_MR_FE(messageOriginal, size)

message_authentifie = dechiffre_RSA_FE(messageChiffre, d, n)

# Affichage des résultats
print("Message original d'Alice à Bob:", messageOriginal)
print("Signature chiffrée par Alice:", messageChiffre)
print("Message authentifié par Bob:", message_authentifie)

OverflowError: Python int too large to convert to C int